In [ ]:
import osiris_utils as ou
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
plt.rcParams['font.size'] = 14
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator,  FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
from scipy import optimize
import contextlib
import io


In [ ]:
# Normalize axis to w_ce

def _set_scaled_formatter(axis, scale_factor, fmt=".2f"):
    axis.set_major_formatter(
        FuncFormatter(lambda v, pos: f"{v*scale_factor:{fmt}}")
    )

def scale_x_ax(scale_factor, fig, ax, label=r"$t[2\pi / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_xlabel(label)
    if lock_ticks:
        # freeze current tick positions
        ticks = ax.get_xticks()
        ax.xaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    return fig, ax

def scale_y_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_ylabel(label)
    if lock_ticks:
        ticks = ax.get_yticks()
        ax.yaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    return fig, ax

def scale_z_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_zlabel(label)
    if lock_ticks:
        ticks = ax.get_zticks()
        ax.zaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax

def scale_3d_axes(scale_factor, fig, ax, fmt=".2f"):
    ax.set_xlabel(r"$x_1[c / \Omega_e]$")
    ax.set_ylabel(r"$x_2[c / \Omega_e]$")
    ax.set_zlabel(r"$x_3[c / \Omega_e]$")
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax


In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, avrg_B, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)

    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        # label1 = ("$t = {:.1f}$".format(t[0]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
        # label2 = ("$t = {:.1f}$".format(t[-1]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
        label1 = None #("$t = {:.1f}$".format(t[0] * avrg_B / 2.0 / np.pi) + r"$ [2\pi / \Omega_e]$")
        label2 = None #("$t = {:.1f}$".format(t[-1] * avrg_B / 2.0 / np.pi)  + r"$[ 2\pi / \Omega_e]$")

    if label == "Baseline":
        color = "black"
        markersize = 0
        linewidth = 0.6
    elif label == r"$m/q=-1$, static":
        linestyle = "--"
        color = "tab:blue"
        markersize = 0.0
        linewidth = 0.8
    elif label == r"$m/q=2$, static":
        linestyle = "--"
        color = "tab:orange"
        markersize = 0.0
        linewidth = 0.8
    else:
        color = None
        markersize = 1.5
        linewidth = 0.8
        linestyle = "-"
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle=linestyle, markersize=markersize, linewidth=linewidth, label=label, color=color)
    
    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x1"]), labelpad=10)
    ax.set_ylabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x2"]), labelpad=10)
    ax.set_zlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x3"]), labelpad=1)
    ax.legend()

    return fig, ax

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt

def plot_particle_line_through_point(
    x0, y0, z0,
    vx, vy, vz,           # constant velocity components
    t_span=(-10.0, 10.0), # parametric time range
    npts=800,             # sampling along the line
    xlim=None,            # (xmin, xmax) or None
    ylim=None,            # (ymin, ymax) or None
    zlim=None,            # (zmin, zmax) or None
    fig=None,
    ax=None,
    start_kwargs=None,    # kwargs for the start marker
    end_kwargs=None,      # kwargs for the end marker
    **plot_kwargs         # forwarded to ax.plot3D
):
    """
    Plot the straight-line trajectory r(t) = r0 + v * t passing through (x0,y0,z0),
    with constant velocity (vx,vy,vz). The curve is clipped to optional spatial limits
    and split into contiguous in-bounds segments to avoid drawing across gaps.
    The first/last markers correspond to the first/last *visible* samples
    inside the provided spatial limits.
    """
    created_ax = False
    if fig is None:
        fig = plt.figure(figsize=(10, 7))
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        created_ax = True
    else:
        old_xlim = ax.get_xlim3d()
        old_ylim = ax.get_ylim3d()
        old_zlim = ax.get_zlim3d()
        ax.set_autoscale_on(False)

    # Parameterization
    t = np.linspace(t_span[0], t_span[1], npts)
    x = x0 + vx * t
    y = y0 + vy * t
    z = z0 + vz * t

    # Default line style
    if "linewidth" not in plot_kwargs and "lw" not in plot_kwargs:
        plot_kwargs["linewidth"] = 1.0
    if "color" not in plot_kwargs:
        plot_kwargs["color"] = "black"
    if "linestyle" not in plot_kwargs and "ls" not in plot_kwargs:
        plot_kwargs["linestyle"] = "-"

    # Build mask for spatial limits
    mask = np.ones_like(x, dtype=bool)
    if xlim is not None:
        mask &= (x >= xlim[0]) & (x <= xlim[1])
    if ylim is not None:
        mask &= (y >= ylim[0]) & (y <= ylim[1])
    if zlim is not None:
        mask &= (z >= zlim[0]) & (z <= zlim[1])

    # Split into contiguous in-bounds runs and plot each segment
    idx = np.where(mask)[0]
    if idx.size > 0:
        breaks = np.where(np.diff(idx) != 1)[0] + 1
        runs = np.split(idx, breaks)
        for run in runs:
            if run.size >= 2:
                ax.plot3D(x[run], y[run], z[run], **plot_kwargs)
            elif run.size == 1:
                # single in-bounds point: still show a dot so it's visible
                ax.scatter(x[run[0]], y[run[0]], z[run[0]], s=10, c=plot_kwargs.get("color", "black"))

        # Highlight first and last *visible* points
        i_first, i_last = idx[0], idx[-1]

        # Defaults for markers
        if start_kwargs is None:
            start_kwargs = dict(s=20, c="lightgreen", edgecolors="none")
        if end_kwargs is None:
            end_kwargs = dict(s=20, c="r", edgecolors="none")

        ax.scatter(x[i_first], y[i_first], z[i_first], **start_kwargs)
        ax.scatter(x[i_last],  y[i_last],  z[i_last],  **end_kwargs)

    # Restore previous limits if using an existing axis
    if not created_ax:
        ax.set_xlim3d(old_xlim)
        ax.set_ylim3d(old_ylim)
        ax.set_zlim3d(old_zlim)

    return fig, ax



import numpy as np
import matplotlib.pyplot as plt

def plot_line_point_velocity2(
    x0, y0, z0, vx, vy, vz,
    t_span=(-10, 10), n=200,
    ax=None, zlim=None,
    color = None, linestyle='-', linewidth=2.0
):
    """
    Plot the line r(t) = r0 + v*t through (x0,y0,z0) with velocity (vx,vy,vz).
    If zlim=(zmin,zmax) is given, only plot samples with zmin <= z <= zmax.
    Does NOT modify axis limits.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    t = np.linspace(t_span[0], t_span[1], n)
    x = x0 + vx*t
    y = y0 + vy*t
    z = z0 + vz*t

    if ax is None:
        fig = plt.figure(figsize=(6, 5))
        ax = fig.add_subplot(111, projection="3d")
    else:
        fig = ax.figure

    # mask by zlim (if provided)
    if zlim is not None:
        zmin, zmax = zlim
        mask = (z >= zmin) & (z <= zmax)
    else:
        mask = np.ones_like(z, dtype=bool)

    idx = np.where(mask)[0]
    if idx.size >= 2:
        ax.plot3D(x[idx], y[idx], z[idx], c=color, linestyle=linestyle, linewidth=linewidth)
        # highlight first/last in-bounds samples
        ax.scatter([x[idx[0]]],  [y[idx[0]]],  [z[idx[0]]],  s=20, c="lightgreen")
        ax.scatter([x[idx[-1]]], [y[idx[-1]]], [z[idx[-1]]], s=20, c="r")
    elif idx.size == 1:
        # only a single in-bounds point; just show it
        ax.scatter([x[idx[0]]], [y[idx[0]]], [z[idx[0]]], s=20)

    return fig, ax






def circle_residuals(params, x, y):
    x_c, y_c, r = params
    return np.sqrt((x - x_c)**2 + (y - y_c)**2) - r


def gyroradius(sim, particle, tmin=0, tmax=-1):
    x = sim["test_electrons"]["tracks"]["x1"][particle, tmin:tmax]
    y = sim["test_electrons"]["tracks"]["x2"][particle, tmin:tmax]
    center_estimate = x.mean(), y.mean(), max(x) - x.mean()
    result = optimize.least_squares(circle_residuals, center_estimate, args=(x, y))
    x_c, y_c, r = result.x

    # --- 1σ error bar for r ---
    J = result.jac                         # N×3 Jacobian at the solution
    res = result.fun                       # residuals (length N)
    dof = max(1, res.size - result.x.size) # N - 3, guard against <=0
    sigma2 = (res @ res) / dof             # residual variance
    JTJ = J.T @ J
    try:
        cov = np.linalg.inv(JTJ) * sigma2
    except np.linalg.LinAlgError:
        cov = np.linalg.pinv(JTJ) * sigma2  # fallback if nearly singular
    r_err = float(np.sqrt(cov[2, 2]))
    print(f"Gyroradius: r = {r:.4f} ± {r_err:.4f} [{sim['test_electrons']['tracks'].units['x1']}]")
    return x_c, y_c


## Gyromotion

In [ ]:
gyroElec = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/Gyration/Boris.in")
gyroProt = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/GyrationProton/Boris.in")
# raw = ou.OsirisRawFile("/home/exxxx5/Tese/Decks/DriftIlustrattions/Gyration/MS/RAW/test_electrons/RAW-test_electrons-000000.h5")
# raw.raw_to_file_tags("/home/exxxx5/Tese/Decks/DriftIlustrattions/Gyration/file_tags2.tags", type="all")

In [ ]:
avrg_B = 2.5

x_e, y_e = gyroradius(gyroElec, particle=0)
x_i, y_i = gyroradius(gyroProt, particle=0)

fig, ax = None, None
fig, ax = plot_trajectory(gyroElec, r"$m/q=-1$", particle=0, avrg_B=avrg_B, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gyroProt, r"$m/q=2$", particle=0, avrg_B=avrg_B, tmax=1000, fig=fig, ax=ax)

fig, ax = plot_particle_line_through_point(
    x_e, y_e, gyroElec["test_electrons"]["tracks"]["x3"][0,0],
    0, 0, 1,          # constant velocity components
    t_span=(0, 1), # parametric time range (arbitrary units)
    npts=800,            # sampling along the line
    xlim=None,           # (xmin, xmax) or None
    ylim=None,           # (ymin, ymax) or None
    zlim=(gyroElec["test_electrons"]["tracks"]["x3"][0,0],gyroElec["test_electrons"]["tracks"]["x3"][0,-1]),           # (zmin, zmax) or None
    fig=fig,
    ax=ax,
    color="tab:blue", 
    linestyle="--", linewidth=2
)

fig, ax = plot_particle_line_through_point(
    x_i, y_i, gyroElec["test_electrons"]["tracks"]["x3"][0,0],
    0, 0, 0.1,          # constant velocity components
    t_span=(0, 1), # parametric time range (arbitrary units)
    npts=800,            # sampling along the line
    xlim=None,           # (xmin, xmax) or None
    ylim=None,           # (ymin, ymax) or None
    zlim=(gyroElec["test_electrons"]["tracks"]["x3"][0,0],gyroElec["test_electrons"]["tracks"]["x3"][0,-1]),           # (zmin, zmax) or None
    fig=fig,
    ax=ax,
    color="tab:orange", 
    linestyle="--", linewidth=2
)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))  # max 4 ticks on x
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

ax.tick_params(axis='z', pad=15)

ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.3f'))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".2f")

# Keep the usual label, but pull it inward and reduce padding
ax.zaxis.set_rotate_label(False)             # stop auto-rotation
ax.zaxis.set_label_coords(0.92, 0.5)         # (x,y) in axis coords; <1.0 pulls it inside

# Give the figure a bit more breathing room

plt.subplots_adjust(right=0.88, top=0.95)  

ax.view_init(elev=20, azim=80)
ax.tick_params(axis='x', pad=-5)
ax.tick_params(axis='y', pad=10)
# ax.tick_params(axis='z', pad=15)
ax.xaxis.labelpad = 1
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 3 
# ax.view_init(elev=15, azim=-110)
ax.legend(loc="upper right")



## EXB


In [ ]:
EXBElec = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/EXBElec/Boris.in")
EXBProt = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/EXBProt/Boris.in")

In [ ]:
gyroElec["test_electrons"]["tracks"]["t"][0, :]

In [ ]:
EXBElec["test_electrons"]["tracks"]["x2"][0, :]

In [ ]:
plt.plot(EXBElec["test_electrons"]["tracks"]["x1"][0, :], EXBElec["test_electrons"]["tracks"]["x2"][0, :])
x = EXBElec["test_electrons"]["tracks"]["x1"][0, :]
# plt.plot(x, 4.7 + 0.048 * (x-EXBElec["test_electrons"]["tracks"]["x1"][0, 0]), linestyle="--", color="red", label=r"$4.7 + 0.05x$")
plt.legend()

In [ ]:
avrg_B = 2.5
# avrg_B = 1
fig, ax = None, None
fig, ax = plot_trajectory(EXBElec, r"$m/q=-1$", particle=0, avrg_B=avrg_B, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(EXBProt, r"$m/q=2$", particle=0, avrg_B=avrg_B, tmax=1000, fig=fig, ax=ax)

ye = (np.max(EXBElec["test_electrons"]["tracks"]["x2"][0, :])+np.min(EXBElec["test_electrons"]["tracks"]["x2"][0, :]))/2
z0 = EXBElec["test_electrons"]["tracks"]["x3"][0,0]
xe = EXBElec["test_electrons"]["tracks"]["x1"][0,0]+0.04
print(xe,ye)

yi = (np.max(EXBProt["test_electrons"]["tracks"]["x2"][0, :])+np.min(EXBProt["test_electrons"]["tracks"]["x2"][0, :]))/2
xi = EXBProt["test_electrons"]["tracks"]["x1"][0,0]+0.08
print(xi,yi)
fig, ax = plot_line_point_velocity2(xe, ye, z0, 1, 0, 0.048, t_span=(0, 10), n=200, ax=ax, zlim=(0,EXBElec["test_electrons"]["tracks"]["x3"][0,-1]), color="tab:blue", linestyle="--")
fig, ax = plot_line_point_velocity2(xi, yi, z0, 1, 0, 0.048, t_span=(0, 10), n=200, ax=ax, zlim=(0,EXBProt["test_electrons"]["tracks"]["x3"][0,-1]), color="tab:orange", linestyle="--")


ax.xaxis.set_major_locator(plt.MaxNLocator(4))  # max 4 ticks on x
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

ax.tick_params(axis='z', pad=15)

ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.3f'))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".2f")

# Keep the usual label, but pull it inward and reduce padding
ax.zaxis.set_rotate_label(False)             # stop auto-rotation
ax.zaxis.set_label_coords(0.92, 0.5)         # (x,y) in axis coords; <1.0 pulls it inside

# Give the figure a bit more breathing room

plt.subplots_adjust(right=0.88, top=0.95)  

# ax.view_init(elev=90, azim=270)
ax.view_init(elev=40, azim=50)
# ax.view_init(elev=0, azim=0)
ax.legend(loc="upper right")






## $\nabla B $ Drift

In [ ]:
GradBElec = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/GradBElec/Boris.in")
GradBProt = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/GradBProt/Boris.in")

In [ ]:
plt.plot(GradBElec["test_electrons"]["tracks"]["x3"][0, :], GradBElec["test_electrons"]["tracks"]["x1"][0, :])
x = GradBElec["test_electrons"]["tracks"]["x3"][0, :]
plt.plot(x, 4.7 +  0.04 * (x-GradBElec["test_electrons"]["tracks"]["x3"][0, 0]), linestyle="--", color="red", label=r"$4.7 + 0.05x$")
plt.legend()

In [ ]:
avrg_B = 5

fig, ax = None, None
fig, ax = plot_trajectory(GradBElec, r"$m/q=-1$", particle=0, avrg_B=avrg_B, tmax=50, fig=fig, ax=ax)
fig, ax = plot_trajectory(GradBProt, r"$m/q=2$", particle=0, avrg_B=avrg_B, tmax=50, fig=fig, ax=ax)

ye = (np.max(GradBElec["test_electrons"]["tracks"]["x2"][0, :])+np.min(GradBElec["test_electrons"]["tracks"]["x2"][0, :]))/2
z0 = GradBElec["test_electrons"]["tracks"]["x3"][0,0]
xe = gyroradius(GradBElec, particle=0, tmax=100)[0]

yi = (np.max(GradBProt["test_electrons"]["tracks"]["x2"][0, :])+np.min(GradBProt["test_electrons"]["tracks"]["x2"][0, :]))/2
xi = gyroradius(GradBProt, particle=0, tmax=100)[0]

fig, ax = plot_line_point_velocity2(xe, ye, z0, 0.04, 0, 1, t_span=(0, 1), n=200, ax=ax, zlim=(0,GradBElec["test_electrons"]["tracks"]["x3"][0,-1]), color="tab:blue", linestyle="--")
fig, ax = plot_line_point_velocity2(xi, yi, z0, -0.08, 0, 1, t_span=(0, 1), n=200, ax=ax, zlim=(0,GradBProt["test_electrons"]["tracks"]["x3"][0,-1]), color="tab:orange", linestyle="--")

ax.xaxis.set_major_locator(plt.MaxNLocator(4))  # max 4 ticks on x
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.3f'))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".2f")

# Keep the usual label, but pull it inward and reduce padding
ax.zaxis.set_rotate_label(False)             # stop auto-rotation
ax.zaxis.set_label_coords(0.8, 0.5)         # (x,y) in axis coords; <1.0 pulls it inside

# ax.set_ylabel('${}$'.format(GradBElec["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(GradBElec["test_electrons"]["tracks"].units["x2"]), labelpad=25)
# ax.set_zlabel('${}$'.format(GradBElec["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(GradBElec["test_electrons"]["tracks"].units["x3"]), labelpad=15)
# # Give the figure a bit more breathing room

ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 30
ax.zaxis.labelpad = 10
plt.subplots_adjust(right=0.88, top=0.95)  

ax.view_init(elev=20, azim=90)

# ax.view_init(elev=0, azim=270)
# ax.view_init(elev=0, azim=90)
ax.legend(loc="upper right")

plt.show()


## Curv Drift

In [ ]:
CurvElec = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/CurvElec/Boris.in")
CurvProt = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/CurvProt/Boris.in")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_field_line_through_point(
    x0, y0, z0,
    Bz=0.0,            # third component of B = (-y, x, Bz)
    n_turns=1,         # how many turns to draw (if limits not given)
    npts=800,          # sampling along the curve
    xlim=(1.2, 4.8),             # tuple (xmin, xmax)
    ylim=(4.5, 6.5),             # tuple (ymin, ymax)
    fig=None,
    ax=None,
    color = "black",
    linestyle='--',
    linewidth=0.8,
    startEndMarkers=False
):
    """
    Plot the field line of B = (-y, x, Bz) passing through (x0, y0, z0).
    If Bz=0 -> circle in the xy-plane. If Bz≠0 -> helix with pitch ~ Bz.
    Optional xlim, ylim restrict where the line is drawn. The curve is
    split into segments so it won't "close" across masked gaps.
    """

    created_ax = False
    if fig is None:
        fig = plt.figure(figsize=(10, 7))
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        created_ax = True
    else:
        old_xlim = ax.get_xlim3d()
        old_ylim = ax.get_ylim3d()
        old_zlim = ax.get_zlim3d()

        # Disable autoscale so adding segments doesn't reset user limits
        ax.set_autoscale_on(False)

    # Parameterization: dx/dt = -y, dy/dt = x, dz/dt = Bz
    r = float(np.hypot(x0, y0))
    phi0 = float(np.arctan2(y0, x0))
    t = np.linspace(0.0, 2*np.pi*n_turns, npts)

    if r == 0.0:
        x = np.zeros_like(t)
        y = np.zeros_like(t)
        z = z0 + Bz * t
    else:
        x = r * np.cos(t + phi0)
        y = r * np.sin(t + phi0)
        z = z0 + Bz * t

    # Build mask for spatial limits
    mask = np.ones_like(x, dtype=bool)
    if xlim is not None:
        mask &= (x >= xlim[0]) & (x <= xlim[1])
    if ylim is not None:
        mask &= (y >= ylim[0]) & (y <= ylim[1])

    # Split into contiguous in-bounds runs to avoid closing across gaps
    idx = np.where(mask)[0]
    if idx.size > 0:
        # break where indices are not consecutive
        breaks = np.where(np.diff(idx) != 1)[0] + 1
        runs = np.split(idx, breaks)
        for run in runs:
            if run.size >= 2:  # need at least two points to draw a segment
                ax.plot3D(x[run], y[run], z[run],
                          linewidth=linewidth, color=color, linestyle=linestyle)

    # Restore previous limits if we were given an existing axis
    if not created_ax:
        ax.set_xlim3d(old_xlim)
        ax.set_ylim3d(old_ylim)
        ax.set_zlim3d(old_zlim)

    return fig, ax


In [ ]:
def plot_field_line_through_pointv2(
    x0, y0, z0,
    Bz=0.0,            # third component of B = (-y, x, Bz)
    n_turns=1,         # how many turns to draw (if limits not given)
    npts=800,          # sampling along the curve
    xlim=(1.2, 4.8),   # tuple (xmin, xmax)
    ylim=(4.5, 6.5),   # tuple (ymin, ymax)
    fig=None,
    ax=None,
    color="black",
    linestyle="--",
    linewidth=0.8,
    startEndMarkers=False,   # if True, mark first/last visible points
    start_kwargs=None,       # kwargs for start marker
    end_kwargs=None,          # kwargs for end marker
    label=None
):
    """
    Plot the field line of B = (-y, x, Bz) passing through (x0, y0, z0).
    If Bz=0 -> circle in the xy-plane. If Bz≠0 -> helix with pitch ~ Bz.
    Optional xlim, ylim restrict where the line is drawn. The curve is
    split into segments so it won't "close" across masked gaps.
    """

    created_ax = False
    if fig is None:
        fig = plt.figure(figsize=(10, 7))
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        created_ax = True
    else:
        old_xlim = ax.get_xlim3d()
        old_ylim = ax.get_ylim3d()
        old_zlim = ax.get_zlim3d()
        # Disable autoscale so adding segments doesn't reset user limits
        ax.set_autoscale_on(False)

    # Parameterization: dx/dt = -y, dy/dt = x, dz/dt = Bz
    r = float(np.hypot(x0, y0))
    phi0 = float(np.arctan2(y0, x0))
    t = np.linspace(0.0, 2*np.pi*n_turns, npts)

    if r == 0.0:
        x = np.zeros_like(t)
        y = np.zeros_like(t)
        z = z0 + Bz * t
    else:
        x = r * np.cos(t + phi0)
        y = r * np.sin(t + phi0)
        z = z0 + Bz * t

    # Build mask for spatial limits
    mask = np.ones_like(x, dtype=bool)
    if xlim is not None:
        mask &= (x >= xlim[0]) & (x <= xlim[1])
    if ylim is not None:
        mask &= (y >= ylim[0]) & (y <= ylim[1])

    # Split into contiguous in-bounds runs to avoid closing across gaps
    idx = np.where(mask)[0]
    if idx.size > 0:
        # Plot each contiguous segment
        breaks = np.where(np.diff(idx) != 1)[0] + 1
        runs = np.split(idx, breaks)
        for run in runs:
            if run.size >= 2:  # need at least two points to draw a segment
                ax.plot3D(
                    x[run], y[run], z[run],
                    linewidth=linewidth, color=color, linestyle=linestyle, label=label
                )
                label = None  # only use label for first segment
            elif run.size == 1:
                # single in-bounds sample: still show a dot so it's visible
                ax.scatter([x[run[0]]], [y[run[0]]], [z[run[0]]], s=10, c=color)

        # Optionally mark the first and last *visible* points
        if startEndMarkers:
            i_start, i_end = idx[0], idx[-1]
            if start_kwargs is None:
                start_kwargs = dict(s=20, c="lightgreen")
            if end_kwargs is None:
                end_kwargs = dict(s=20, c="r")
            ax.scatter([x[i_start]], [y[i_start]], [z[i_start]], **start_kwargs)
            ax.scatter([x[i_end]],   [y[i_end]],   [z[i_end]],   **end_kwargs)

    # Restore previous limits if we were given an existing axis
    if not created_ax:
        ax.set_xlim3d(old_xlim)
        ax.set_ylim3d(old_ylim)
        ax.set_zlim3d(old_zlim)

    return fig, ax

import numpy as np
import matplotlib.pyplot as plt

def plot_xy_with_vertical_velocity(x, y, vz, z0=0.0, ax=None, zlim=None,
                                   color="black", lw=1.0):
    """
    Plot a 3D polyline at fixed (x[i], y[i]) samples with a vertical velocity vz:
        z[i] = z0 + vz * i   (unit time step)
    Marks the first and last *in-bounds* points if zlim is given.
    Does not change existing axis limits.
    """
    x = np.asarray(x); y = np.asarray(y)
    assert x.shape == y.shape, "x and y must have the same shape"
    n = x.size
    z = z0 + vz * np.arange(n, dtype=float)

    if ax is None:
        fig = plt.figure(figsize=(6, 5))
        ax = fig.add_subplot(111, projection="3d")
    else:
        fig = ax.figure

    # Mask by z-range if provided
    if zlim is not None:
        zmin, zmax = zlim
        mask = (z >= zmin) & (z <= zmax)
    else:
        mask = np.ones(n, dtype=bool)

    idx = np.where(mask)[0]
    if idx.size >= 2:
        ax.plot3D(x[idx], y[idx], z[idx], color=color, lw=lw)
        # first / last visible points
        ax.scatter([x[idx[0]]],  [y[idx[0]]],  [z[idx[0]]],  s=20, c="lightgreen")
        ax.scatter([x[idx[-1]]], [y[idx[-1]]], [z[idx[-1]]], s=20, c="r")
    elif idx.size == 1:
        ax.scatter([x[idx[0]]], [y[idx[0]]], [z[idx[0]]], s=30, c=color)

    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    return fig, ax


In [ ]:
avrg_B = 4
fig, ax = None, None
tmax=49.6
tmax=50
fig, ax = plot_trajectory(CurvElec, r"$m/q=-1$", particle=0, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
# fig, ax = plot_trajectory(CurvProt, r"$m/q=2$", particle=0, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)

fig, ax = plot_field_line_through_pointv2(CurvElec["test_electrons"]["tracks"]["x1"][0, 0], 
                                        CurvElec["test_electrons"]["tracks"]["x2"][0, 0], 
                                        CurvElec["test_electrons"]["tracks"]["x3"][0, 0]-0.018, 
                                        Bz=0, n_turns=1, fig=fig, ax=ax, linewidth=2, linestyle="-", label="Field line")

fig, ax = plot_field_line_through_pointv2(CurvElec["test_electrons"]["tracks"]["x1"][0, 0], 
                                        CurvElec["test_electrons"]["tracks"]["x2"][0, 0], 
                                        CurvElec["test_electrons"]["tracks"]["x3"][0, 0]-0.018, 
                                        Bz=-0.018, n_turns=0.35, fig=fig, ax=ax, startEndMarkers=True, color="tab:blue", linewidth=2.0,    
                                        xlim=(CurvElec["test_electrons"]["tracks"]["x1"][0, 500]-0.00000001, CurvElec["test_electrons"]["tracks"]["x1"][0, 0]),   # tuple (xmin, xmax)
                                        ylim=(CurvElec["test_electrons"]["tracks"]["x2"][0, 0]-0.00000001, CurvElec["test_electrons"]["tracks"]["x2"][0, 509]),   # tuple (ymin, ymax)
                                        )

# fig, ax = plot_field_line_through_pointv2(CurvElec["test_electrons"]["tracks"]["x1"][0, 0], 
#                                         CurvElec["test_electrons"]["tracks"]["x2"][0, 0], 
#                                         CurvElec["test_electrons"]["tracks"]["x3"][0, 0]+0.036, 
#                                         Bz=0.036, n_turns=0.35, fig=fig, ax=ax, startEndMarkers=True, color="tab:orange", linewidth=2.0,    
#                                         # xlim=(CurvElec["test_electrons"]["tracks"]["x1"][0, 500], CurvElec["test_electrons"]["tracks"]["x1"][0, 0]),   # tuple (xmin, xmax)
#                                         ylim=(CurvElec["test_electrons"]["tracks"]["x2"][0, 0]-0.00000001, CurvElec["test_electrons"]["tracks"]["x2"][0, 509]),   # tuple (ymin, ymax)
#                                         )


ax.xaxis.set_major_locator(plt.MaxNLocator(4))  # max 4 ticks on x
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

ax.tick_params(axis='y', pad=10)
ax.tick_params(axis='z', pad=15)

ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.3f'))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".2f")

# Keep the usual label, but pull it inward and reduce padding
ax.zaxis.set_rotate_label(False)             # stop auto-rotation
ax.zaxis.set_label_coords(0.92, 0.5)         # (x,y) in axis coords; <1.0 pulls it inside

plt.subplots_adjust(right=0.88, top=0.95)  
ax.yaxis.labelpad = 25
# ax.view_init(elev=90, azim=270)
ax.view_init(elev=15, azim=80)

# ax.view_init(elev=0, azim=90)
ax.legend(loc="upper right", bbox_to_anchor=(1, 0.8))
ax.set_zlabel("")
ax.text2D(0.07, 0.66, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()


## Mirror Force

In [ ]:
MirrorElec = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/MirrorElec/Boris.in")
MirrorProt = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/MirrorProt/Boris.in")

In [ ]:
print("p3", MirrorElec["test_electrons"]["tracks"]["p3"][0, 0], MirrorElec["test_electrons"]["tracks"]["p3"][0, -1])
# print("B3", MirrorElec["test_electrons"]["tracks"]["x3"][0, 0]-3.0, MirrorElec["test_electrons"]["tracks"]["x3"][0, -1]-3.0)

In [ ]:
# print("x", MirrorElec["test_electrons"]["tracks"][["x1", "x2", "x3"]][0, 0])

In [ ]:
fig, ax = plt.subplots()

ax.plot(MirrorElec["test_electrons"]["tracks"]["t"][0, :], (MirrorElec["test_electrons"]["tracks"]["p1"][0, :]), label=r"$p_1$")
ax.plot(MirrorElec["test_electrons"]["tracks"]["t"][0, :], (MirrorElec["test_electrons"]["tracks"]["p2"][0, :]), label=r"$p_2$")
ax.plot(MirrorElec["test_electrons"]["tracks"]["t"][0, :], (MirrorElec["test_electrons"]["tracks"]["p3"][0, :]), label=r"$p_3$")
ax.plot(MirrorElec["test_electrons"]["tracks"]["t"][0, :], np.sqrt(MirrorElec["test_electrons"]["tracks"]["p1"][0, :]**2 + MirrorElec["test_electrons"]["tracks"]["p2"][0, :]**2 + MirrorElec["test_electrons"]["tracks"]["p3"][0, :]**2), label=r"$p_{total}$")
ax.set_xlabel('${}$'.format(MirrorElec["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(MirrorElec["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel(r"$p$" + " [{}]".format(MirrorElec["test_electrons"]["tracks"].units["p2"]))
# ax.set_ylim(0, 0.015)
# ax.set_yscale("log")
ax.legend()
plt.show()


In [ ]:
# fig, ax = None, None
# fig, ax = plot_trajectory(MirrorElec, r"$m/q=-1$", particle=0, tmax=150, fig=fig, ax=ax)
# # fig, ax = plot_trajectory(CurvProt, r"$m/q=2$", particle=0, tmax=150, fig=fig, ax=ax)

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))  # max 4 ticks on x
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))
# ax.zaxis.set_major_locator(plt.MaxNLocator(4))

# ax.tick_params(axis='y', pad=15)
# ax.tick_params(axis='z', pad=25)

# ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
# ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.3f'))

# # Keep the usual label, but pull it inward and reduce padding
# ax.zaxis.set_rotate_label(False)             # stop auto-rotation
# ax.zaxis.set_label_coords(0.52, 0.5)         # (x,y) in axis coords; <1.0 pulls it inside

# ax.set_ylabel('${}$'.format(MirrorElec["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(MirrorElec["test_electrons"]["tracks"].units["x2"]), labelpad=25)
# ax.set_zlabel('${}$'.format(MirrorElec["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(MirrorElec["test_electrons"]["tracks"].units["x3"]), labelpad=10)
# # Give the figure a bit more breathing room

# plt.subplots_adjust(right=0.88, top=0.95)  

# # ax.view_init(elev=0, azim=270)
# ax.view_init(elev=15, azim=115)
# ax.legend(loc="upper right")

# plt.show()


In [ ]:
import csv

x = []
y = []
ux = []
with open("/home/exxxx5/Tese/Decks/DriftIlustrattions/mirrorTraj.csv", newline="") as f:
    reader = csv.DictReader(f)  # uses header "x","y"
    for row in reader:
        x.append(float(row["x"]))
        y.append(float(row["y"]))
        ux.append(float(row["ux"]))

# x_elec and y_elec are now lists of floats


In [ ]:
ux
def first_negative_idx(lst):
    return next((i for i, v in enumerate(lst) if v < 0), None)

idx = first_negative_idx(ux)  # e.g., your list
if idx is not None:
    print("first negative at index", idx, "value", ux[idx])
else:
    print("no negatives")


In [ ]:
300 * 10 * 0.013804693251470665

In [ ]:
avrg_B = 2.42581
print(9*avrg_B)

print(0.25*avrg_B)

In [ ]:
# dt = 0.013804693251470665
avrg_B = 2.42581
fig, ax =plt.subplots()

label1 = None
label2 = None
# if ax is None:
#     ax = fig.add_subplot(111, projection='3d')
#     label1 = ("$t = {:.1f}$".format(t[0]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
#     label2 = ("$t = {:.1f}$".format(t[-1]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))


color = None
markersize = 1.5
linewidth = 0.8


# Highlight start and end points
t =[0, 41.41407975441199]
ymean= np.mean(y)
label1 = ("$t = {:.1f}$".format(t[0] * avrg_B / 2.0 / np.pi) + r"$ [2\pi / \Omega_e]$")
label2 = ("$t = {:.1f}$".format(t[-1] * avrg_B / 2.0 / np.pi)  + r"$[ 2\pi / \Omega_e]$")
ax.scatter(x[0], y[0], color='lightgreen', s=20, label=label1)
ax.scatter(x[300], y[300], color='r', s=20, label=label2)
ax.scatter(x[0], ymean, color='lightgreen', s=20)
ax.scatter(x[300], ymean, color='r', s=20)

# Plot the trajectory
ax.plot(x[:220], y[:220], marker='o', linestyle='-', markersize=markersize, linewidth=linewidth, label=r"$p_1>0$", color=color)
ax.plot(x[212:301], y[212:301], marker='o', linestyle='-', markersize=markersize, linewidth=linewidth, label=r"$p_1<0$", color=color)


ax.plot(x[:220], np.mean(y)+0*np.array(x[:220]), c="tab:blue", linestyle="--", linewidth=2)
ax.plot(x[212:301], np.mean(y)+0*np.array(x[212:301]), c="tab:orange", linestyle="--", linewidth=2)
# Labels and title
# ax.set_xlabel('${}$'.format(MirrorElec["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(MirrorElec["test_electrons"]["tracks"].units["x1"]))
# ax.set_ylabel('${}$'.format(MirrorElec["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(MirrorElec["test_electrons"]["tracks"].units["x2"]))

scale_x_ax(avrg_B, fig, ax, label=r"$x_1[c/\Omega_e]$")
scale_y_ax(avrg_B, fig, ax, label=r"$x_2[c/\Omega_e]$")
# ax.set_zlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x3"]), labelpad=1)
ax.set_xlim(4.4, 5.5)
ax.legend(loc="lower left")


## Polarization

In [ ]:
def plot_line_point_velocity2(
    x0, y0, z0,
    vx, vy, vz,
    t_span=(-10, 10), n=200,
    ax=None, zlim=None,
    color=None, linestyle='-', linewidth=2.0,
    vx0=0.0
):
    """
    Plot a trajectory with:
      v_y = vy (constant), v_z = vz (constant),
      v_x(t) = vx * t  (i.e., 'constant * t' in x).
    This gives positions:
      x(t) = x0 + vx0*t + 0.5*vx*t^2
      y(t) = y0 + vy*t
      z(t) = z0 + vz*t

    If zlim=(zmin,zmax) is given, only plot samples with zmin <= z <= zmax.
    Does NOT modify axis limits.

    Parameters
    ----------
    vx : float
        Coefficient for the time-dependent x-velocity v_x(t) = vx * t.
    vx0 : float, optional
        (Optional) constant x-velocity offset so v_x(t) = vx0 + vx*t.
        Defaults to 0.0.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    t = np.linspace(t_span[0], t_span[1], n)

    # Positions from the specified velocities
    x = x0 + vx0*t + 0.5*vx*(t**2)   # integrate v_x(t) = vx0 + vx*t
    y = y0 + vy*t
    z = z0 + vz*t

    if ax is None:
        fig = plt.figure(figsize=(6, 5))
        ax = fig.add_subplot(111, projection="3d")
    else:
        fig = ax.figure

    # mask by zlim (if provided)
    if zlim is not None:
        zmin, zmax = zlim
        mask = (z >= zmin) & (z <= zmax)
    else:
        mask = np.ones_like(z, dtype=bool)

    idx = np.where(mask)[0]
    if idx.size >= 2:
        ax.plot3D(x[idx], y[idx], z[idx], c=color, linestyle=linestyle, linewidth=linewidth)
        # highlight first/last in-bounds samples
        ax.scatter([x[idx[0]]],  [y[idx[0]]],  [z[idx[0]]],  s=20, c="lightgreen")
        ax.scatter([x[idx[-1]]], [y[idx[-1]]], [z[idx[-1]]], s=20, c="r")
    elif idx.size == 1:
        ax.scatter([x[idx[0]]], [y[idx[0]]], [z[idx[0]]], s=20, c=color if color is not None else "k")

    return fig, ax


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_line_point_velocity_times(
    x0, y0, z0,
    vx, vy, vz,          # v_y, v_z constant; v_x(t) = vx0 + vx*t
    t,                   # 1D array-like of times
    ax=None, zlim=None,
    color=None, linestyle='-', linewidth=2.0,
    vx0=0.0
):
    """
    Plot a trajectory parameterized by an explicit time vector `t` with:
      v_y = vy (constant), v_z = vz (constant),
      v_x(t) = vx0 + vx * t  ->  x(t) = x0 + vx0*t + 0.5*vx*t^2

    If zlim=(zmin,zmax) is given, only plot samples with zmin <= z <= zmax.
    Does NOT modify existing axis limits. Marks first/last in-bounds points.
    """
    t = np.asarray(t, dtype=float).ravel()

    x = x0 + vx0*t + 0.5*vx*(t**2)
    y = y0 + vy*t
    z = z0 + vz*t

    if ax is None:
        fig = plt.figure(figsize=(6, 5))
        ax = fig.add_subplot(111, projection="3d")
    else:
        fig = ax.figure

    # mask by zlim (if provided)
    if zlim is not None:
        zmin, zmax = zlim
        mask = (z >= zmin) & (z <= zmax)
    else:
        mask = np.ones_like(z, dtype=bool)

    idx = np.where(mask)[0]
    if idx.size >= 2:
        ax.plot3D(x[idx], y[idx], z[idx], c=color, linestyle=linestyle, linewidth=linewidth)
        # highlight first/last in-bounds samples
        ax.scatter([x[idx[0]]],  [y[idx[0]]],  [z[idx[0]]],  s=20, c="lightgreen")
        ax.scatter([x[idx[-1]]], [y[idx[-1]]], [z[idx[-1]]], s=20, c="r")
    elif idx.size == 1:
        ax.scatter([x[idx[0]]], [y[idx[0]]], [z[idx[0]]], s=20, c=color if color else "k")
    # else: nothing in-bounds -> plot nothing

    return fig, ax


In [ ]:
PolarElec = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/PolarElec/Boris.in")
PolarProt = ou.Simulation("/home/exxxx5/Tese/Decks/DriftIlustrattions/PolarProt/Boris.in")

In [ ]:
avrg_B = 2.5

fig, ax = None, None
fig, ax = plot_trajectory(PolarElec, r"$m/q=-1$", avrg_B=avrg_B, particle=0, tmax=50, fig=fig, ax=ax)
# fig, ax = plot_trajectory(gyroElec, r"$m/q=-1$, static", particle=0, tmax=50, fig=fig, ax=ax)
fig, ax = plot_trajectory(PolarProt, r"$m/q=2$", avrg_B=avrg_B, particle=0, tmax=50, fig=fig, ax=ax)
# fig, ax = plot_trajectory(gyroProt, r"$m/q=2$, static", particle=0, tmax=50, fig=fig, ax=ax)

xe= PolarElec["test_electrons"]["tracks"]["x1"][0,0]
ye = PolarElec["test_electrons"]["tracks"]["x2"][0,0]
z0 = PolarElec["test_electrons"]["tracks"]["x3"][0,0]


# v_y = 0.1, v_z = -0.05 (constants)
# v_x(t) = 0.02 * t  (quadratic x)
# optional constant x-velocity offset vx0 = 0.3 -> v_x(t)=0.3 + 0.02 t
fig, ax = plot_line_point_velocity_times(xe, ye, z0, vx=0.002, vy=-8e-4, vz=0.01, t=PolarElec["test_electrons"]["tracks"]["t"][0,:],
                                    vx0=0, ax=ax,linestyle="--", color="tab:blue", linewidth=2)

fig, ax = plot_line_point_velocity_times(xe, ye, z0, vx=0.002, vy=2*8e-4, vz=0.01, t=PolarElec["test_electrons"]["tracks"]["t"][0,:],
                                    vx0=0, ax=ax,linestyle="--", color="tab:orange", linewidth=2)

# fig, ax = plot_line_point_velocity2(xe, ye, z0, 0.03, 0.005, 0.01, t_span=(0, 10), n=200, ax=ax, zlim=(0,PolarElec["test_electrons"]["tracks"]["x3"][0,-1]), color="tab:blue", linestyle="--")
# fig, ax = plot_line_point_velocity2(xi, yi, z0, 1, 0, 0.048, t_span=(0, 10), n=200, ax=ax, zlim=(0,EXBPPolarElecrot["test_electrons"]["tracks"]["x3"][0,-1]), color="tab:orange", linestyle="--")



ax.xaxis.set_major_locator(plt.MaxNLocator(4))  # max 4 ticks on x
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='z', pad=15)

ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.3f'))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".2f")

# Keep the usual label, but pull it inward and reduce padding
ax.zaxis.set_rotate_label(False)             # stop auto-rotation
ax.zaxis.set_label_coords(0.92, 0.5)         # (x,y) in axis coords; <1.0 pulls it inside
ax.xaxis.labelpad = 25
# ax.set_xlabel('${}$'.format(PolarElec["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(PolarElec["test_electrons"]["tracks"].units["x1"]), labelpad=25)
# ax.set_ylabel('${}$'.format(PolarElec["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(PolarElec["test_electrons"]["tracks"].units["x2"]), labelpad=5)
# ax.set_zlabel('${}$'.format(PolarElec["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(PolarElec["test_electrons"]["tracks"].units["x3"]), labelpad=1)
# Give the figure a bit more breathing room

plt.subplots_adjust(right=0.88, top=0.95)  

ax.view_init(elev=20, azim=170)
# ax.view_init(elev=0, azim=-90)
ax.legend(loc="upper left")

plt.show()
